# Facial expression detection

## Implementation

#### Import libraries

In [4]:
import numpy as np
import cv2
import tensorflow
import face_recognition

from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import model_from_json

### Video from real time webcam

In [ ]:
# Capture the video from default camera
webcam_video_stream = cv2.VideoCapture(0)

# Face expression model initialization
face_exp_model = model_from_json(open("./Face Recognition/Models/Facial expression model/facial_expression_model_structure.json", "r").read())
# Load weights into model
face_exp_model.load_weights("./Face Recognition/Models/Facial expression model/facial_expression_model_weights.h5")
# List of emotions labels
emotions_label = ("angry", "disgust", "fear", "happy", "sad", "surprise", "neutral")

# Initialize empty array for face locations
all_face_locations = []

# Loop through every frame in the video
while True:
    # Get the current frame from the video stream as an image
    ret, current_frame = webcam_video_stream.read()
    # Resize the current frame to 1/4 size to proces faster
    current_frame_small = cv2.resize(current_frame, (0, 0), fx = 0.25, fy = 0.25)
    # Detect all faces in the image
    all_face_locations = face_recognition.face_locations(current_frame_small, 2, model = "hog")

    # Looping through the face locations
    for index, current_face_location in enumerate(all_face_locations):
        top_position, right_position, bottom_position, left_position = current_face_location
        # Change the position magnitude to fit the actual size video frame
        top_position = top_position * 4
        right_position = right_position * 4
        bottom_position = bottom_position * 4
        left_position = left_position * 4
        # Printing the location of current face
        print("Found face {} at top: {}, right: {}, bottom: {}, left: {}".format(index + 1, top_position, right_position, bottom_position, left_position))
        # Extract the face from the frame, blur it, paste it back to the frame slicing the current face from main image
        current_face_image = current_frame[top_position:bottom_position, left_position:right_position]      
        # Draw rectangle around the face detected
        cv2.rectangle(current_frame, (left_position, top_position), (right_position, bottom_position), (0, 0, 255), 2)

        # Preprocess input, convert it to an image like as the data in dataset
        # Convert to grayscale
        current_face_image = cv2.cvtColor(current_face_image, cv2.COLOR_BGR2GRAY)
        # Resize to 48 x 48 px size
        current_face_image = cv2.resize(current_face_image, (48, 48))
        # Convert the PIL image into a 3d numpy array
        img_pixels = image.img_to_array(current_face_image)
        # Expand the shape of an array into single row multiple columns
        img_pixels = np.expand_dims(img_pixels, axis = 0)
        # Pixels are in rage of [0, 255]. Normalize all pixels in scale of [0, 1]
        img_pixels /= 255
        # Do prediction using model, get the prediction values for all 7 expressions
        exp_predictions = face_exp_model.predict(img_pixels)
        # Find max indexed prediction value (0 till 7)
        max_index = np.argmax(exp_predictions[0])
        # Get corresponding labels from emotions_labels
        emotion_label = emotions_label[max_index]
        # Display the name as text in the image
        font = cv2.FONT_HERSHEY_DUPLEX
        cv2.putText(current_frame, emotion_label, (left_position, bottom_position), font, 0.5, (255,255,255),1)

    # Showing the current fae with rectangle drawn
    cv2.imshow("Webcam Video", current_frame)
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

webcam_video_stream.release()
cv2.destroyAllWindows() 